# Collaboration and Competition

---

In this notebook, you will learn how to use the Unity ML-Agents environment for the third project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893) program.

### 1. Start the Environment

We begin by importing the necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [1]:
from unityagents import UnityEnvironment
import numpy as np

Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Tennis.app"`
- **Windows** (x86): `"path/to/Tennis_Windows_x86/Tennis.exe"`
- **Windows** (x86_64): `"path/to/Tennis_Windows_x86_64/Tennis.exe"`
- **Linux** (x86): `"path/to/Tennis_Linux/Tennis.x86"`
- **Linux** (x86_64): `"path/to/Tennis_Linux/Tennis.x86_64"`
- **Linux** (x86, headless): `"path/to/Tennis_Linux_NoVis/Tennis.x86"`
- **Linux** (x86_64, headless): `"path/to/Tennis_Linux_NoVis/Tennis.x86_64"`

For instance, if you are using a Mac, then you downloaded `Tennis.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Tennis.app")
```

In [2]:
env = UnityEnvironment(file_name="C:/Users/User/deep-reinforcement-learning/p3_collab-compet/Tennis_Windows_x86_64/Tennis.exe")

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: TennisBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 8
        Number of stacked Vector Observation: 3
        Vector Action space type: continuous
        Vector Action space size (per agent): 2
        Vector Action descriptions: , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [3]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

In this environment, two agents control rackets to bounce a ball over a net. If an agent hits the ball over the net, it receives a reward of +0.1.  If an agent lets a ball hit the ground or hits the ball out of bounds, it receives a reward of -0.01.  Thus, the goal of each agent is to keep the ball in play.

The observation space consists of 8 variables corresponding to the position and velocity of the ball and racket. Two continuous actions are available, corresponding to movement toward (or away from) the net, and jumping. 

Run the code cell below to print some information about the environment.

In [5]:
import torch
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents 
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for both agent looks like:',states)
print('The state for the first agent looks like:', states[0])
print('The state for the second agent looks like:', states[1])
#print(torch.from_numpy(states).float().unsqueeze(0).shape)
print(torch.from_numpy(states).view(1,-1))

Number of agents: 2
Size of each action: 2
There are 2 agents. Each observes a state with length: 24
The state for both agent looks like: [[ 0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.         -6.65278625 -1.5
  -0.          0.          6.83172083  6.         -0.          0.        ]
 [ 0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.         -6.4669857  -1.5
   0.          0.         -6.83172083  6.          0.          0.        ]]
The state for the first agent looks like: [ 0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.         -6.65278625 -1.5
 -0.          0.          6.83172083  6.         -0.          0.        ]
T

### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agents and receive feedback from the environment.

Once this cell is executed, you will watch the agents' performance, if they select actions at random with each time step.  A window should pop up that allows you to observe the agents.

Of course, as part of the project, you'll have to change the code so that the agents are able to use their experiences to gradually choose better actions when interacting with the environment!

In [36]:
from collections import namedtuple, deque
import random

class ReplayBuffer:
    """Fixed-size buffer to store experience tuples."""

    def __init__(self, action_size, buffer_size, batch_size, seed):
        """Initialize a ReplayBuffer object.
        Params
        ======
            buffer_size (int): maximum size of buffer
            batch_size (int): size of each training batch
        """
        self.action_size = action_size
        self.memory = deque(maxlen=buffer_size)  # internal memory (deque)
        self.batch_size = batch_size
        self.experience = namedtuple("Experience", field_names=["state", "action", "reward", "next_state", "done"])
        self.seed = random.seed(seed)
    
    def add(self, state, action, reward, next_state, done):
        """Add a new experience to memory."""
        e = self.experience(state, action, reward, next_state, done)
        self.memory.append(e)
    
    def sample(self):
        """Randomly sample a batch of experiences from memory."""
        experiences = random.sample(self.memory, k=self.batch_size)

        states = torch.from_numpy(np.vstack([e.state for e in experiences if e is not None])).float().to(device)
        actions = torch.from_numpy(np.vstack([e.action for e in experiences if e is not None])).float().to(device)
        rewards = torch.from_numpy(np.vstack([e.reward for e in experiences if e is not None])).float().to(device)
        next_states = torch.from_numpy(np.vstack([e.next_state for e in experiences if e is not None])).float().to(device)
        dones = torch.from_numpy(np.vstack([e.done for e in experiences if e is not None]).astype(np.uint8)).float().to(device)

        return (states, actions, rewards, next_states, dones)

    def __len__(self):
        """Return the current size of internal memory."""
        return len(self.memory)

action_size=2
BUFFER_SIZE=100
BATCH_SIZE=2
random_seed=6
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
memory = ReplayBuffer(action_size, BUFFER_SIZE, BATCH_SIZE, random_seed)

    
for i in range(1, 2):                                      # play game for 5 episodes
    env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
    states = env_info.vector_observations                  # get the current state (for each agent)
    #print(states)
    #print(states.shape)
    #print(type(states))
    scores = np.zeros(num_agents)                          # initialize the score (for each agent)
    #print(scores)
    while True:
        actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
        print(actions)
        actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
        print(actions)
        print(actions.shape)
        env_info = env.step(actions)[brain_name]           # send all actions to tne environment
        next_states = env_info.vector_observations         # get next state (for each agent)
        #print(next_states)
        rewards = env_info.rewards                         # get reward (for each agent)
        #print(rewards)
        #print(type(rewards))
        dones = env_info.local_done                        # see if episode finished
        #print(dones)
        scores += env_info.rewards                         # update the score (for each agent)
        memory.add(states, actions, rewards, next_states, dones)
        states = next_states                               # roll over states to next time step
        if np.any(dones):                                  # exit loop if episode finished
            break
    print('Score (max over agents) from episode {}: {}'.format(i, np.max(scores)))

experiences = memory.sample()
states, actions, rewards, next_states, dones = experiences
#print(states)
#print(states.shape)
#print(states[:,:24])
#print(states[24:48])
#print(actions)
#print(actions.shape)
#print(rewards)

[[-0.71739807 -1.51871428]
 [-0.93668207 -0.42364638]]
[[-0.71739807 -1.        ]
 [-0.93668207 -0.42364638]]
(2, 2)
[[ 0.27675477 -0.32105588]
 [ 0.15333184  1.34192422]]
[[ 0.27675477 -0.32105588]
 [ 0.15333184  1.        ]]
(2, 2)
[[ 0.74052869 -1.33018516]
 [-0.2604412   1.30094784]]
[[ 0.74052869 -1.        ]
 [-0.2604412   1.        ]]
(2, 2)
[[ 0.15704474  1.94754446]
 [ 0.18946547 -0.96539862]]
[[ 0.15704474  1.        ]
 [ 0.18946547 -0.96539862]]
(2, 2)
[[ 0.74107074  0.5838909 ]
 [-1.56078164 -0.7419027 ]]
[[ 0.74107074  0.5838909 ]
 [-1.         -0.7419027 ]]
(2, 2)
[[0.78033126 0.69603176]
 [1.45640484 0.77507587]]
[[0.78033126 0.69603176]
 [1.         0.77507587]]
(2, 2)
[[1.06994087 1.800069  ]
 [1.73579506 0.03290513]]
[[1.         1.        ]
 [1.         0.03290513]]
(2, 2)
[[-0.02444485 -0.97984579]
 [-0.56229963 -0.47194977]]
[[-0.02444485 -0.97984579]
 [-0.56229963 -0.47194977]]
(2, 2)
[[-1.1931426  -1.1548471 ]
 [-2.72515685  0.51400298]]
[[-1.         -1.        

When finished, you can close the environment.

In [ ]:
env.close()

### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```

In [4]:
####### Model DDPG #########

import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import random
import copy
from collections import namedtuple, deque
import matplotlib.pyplot as plt
import pandas as pd


import torch.optim as optim

def hidden_init(layer):
    fan_in = layer.weight.data.size()[0]
    lim = 1. / np.sqrt(fan_in)
    return (-lim, lim)

class Actor(nn.Module):
    """Actor (Policy) Model."""

    def __init__(self, state_size, action_size, seed, fc1_units=240, fc2_units=240):
        """Initialize parameters and build m/odel.
        Params
        ======
            state_size (int): Dimension of each state
            action_size (int): Dimension of each action
            seed (int): Random seed
            fc1_units (int): Number of nodes in first hidden layer
            fc2_units (int): Number of nodes in second hidden layer
        """
        super(Actor, self).__init__()
        self.seed = torch.manual_seed(seed)
        self.fc1 = nn.Linear(state_size, fc1_units)
        self.fc1_bn=nn.BatchNorm1d(fc1_units)
        self.fc2 = nn.Linear(fc1_units, fc2_units)
        self.fc2_bn=nn.BatchNorm1d(fc2_units)
        self.fc3 = nn.Linear(fc2_units, action_size)
        self.reset_parameters()
        
    def reset_parameters(self):
        self.fc1.weight.data.uniform_(*hidden_init(self.fc1))
        self.fc2.weight.data.uniform_(*hidden_init(self.fc2))
        self.fc3.weight.data.uniform_(-3e-3, 3e-3)

    def forward(self, state):
        """Build an actor (policy) network that maps states -> actions."""
        #x=self.fc1(state)
        #x = F.relu(self.fc1_bn(x))
        x=F.relu(self.fc1(state))
        #x=self.fc2(x)
        #x = F.relu(self.fc2_bn(x))
        x=F.relu(self.fc2(x))
        return F.tanh(self.fc3(x))


class Critic(nn.Module):
    """Critic (Value) Model."""

    def __init__(self, state_size, action_size, seed, fcs1_units=480, fc2_units=480):
        """Initialize parameters and build model.
        Params
        ======
            state_size (int): Dimension of each state
            action_size (int): Dimension of each action
            seed (int): Random seed
            fcs1_units (int): Number of nodes in the first hidden layer
            fc2_units (int): Number of nodes in the second hidden layer
        """
        super(Critic, self).__init__()
        self.seed = torch.manual_seed(seed)
        self.fcs1 = nn.Linear(state_size*2, fcs1_units)
        self.fc2 = nn.Linear(fcs1_units+(action_size*2), fc2_units)
        self.fc3 = nn.Linear(fc2_units, 1)
        self.reset_parameters()

    def reset_parameters(self):
        self.fcs1.weight.data.uniform_(*hidden_init(self.fcs1))
        self.fc2.weight.data.uniform_(*hidden_init(self.fc2))
        self.fc3.weight.data.uniform_(-3e-3, 3e-3)

    def forward(self, state, action):
        """Build a critic (value) network that maps (state, action) pairs -> Q-values."""
        xs = F.relu(self.fcs1(state))
        x = torch.cat((xs, action), dim=1)
        x = F.relu(self.fc2(x))
        return self.fc3(x)

In [15]:
#########  Agent DDPG #########

BUFFER_SIZE = int(1e5)  # replay buffer size
BATCH_SIZE = 128        # minibatch size
GAMMA = 0.99            # discount factor
TAU = 1e-3             # for soft update of target parameters
LR_ACTOR = 4e-4         # learning rate of the actor 
LR_CRITIC = 4e-4        # learning rate of the critic
WEIGHT_DECAY = 0        # L2 weight decay
learn_step_=4

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

class Agent():
    """Interacts with and learns from the environment."""
    
    def __init__(self, state_size, action_size, random_seed):
        """Initialize an Agent object.
        
        Params
        ======
            state_size (int): dimension of each state
            action_size (int): dimension of each action
            random_seed (int): random seed
        """
        self.state_size = state_size
        self.action_size = action_size
        self.seed = random.seed(random_seed)
        self.learn_step=0
        
        # Actor Network (w/ Target Network)
        self.actor_local_first = Actor(state_size, action_size, random_seed).to(device)
        self.actor_target_first = Actor(state_size, action_size, random_seed).to(device)
        self.actor_optimizer_first = optim.Adam(self.actor_local_first.parameters(), lr=LR_ACTOR)
        self.actor_local_second = Actor(state_size, action_size, random_seed).to(device)
        self.actor_target_second = Actor(state_size, action_size, random_seed).to(device)
        self.actor_optimizer_second = optim.Adam(self.actor_local_second.parameters(), lr=LR_ACTOR)

        # Critic Network (w/ Target Network)
        self.critic_local = Critic(state_size, action_size, random_seed).to(device)
        self.critic_target = Critic(state_size, action_size, random_seed).to(device)
        self.critic_optimizer = optim.Adam(self.critic_local.parameters(), lr=LR_CRITIC, weight_decay=WEIGHT_DECAY)

        # Noise process
        self.noise = OUNoise(action_size, random_seed)
        
        # Replay memory
        self.memory = ReplayBuffer(action_size, BUFFER_SIZE, BATCH_SIZE, random_seed)
    
    def step(self, state, action, reward, next_state, done,learntimes=1):
        """Save experience in replay memory, and use random sample from buffer to learn."""
        # Save experience / reward
        state_add=state.reshape(1,-1)
        action_add=action.reshape(1,-1)
        reward_add=reward
        next_state_add=next_state.reshape(1,-1)
        done_add=done
        self.memory.add(state_add, action_add, reward_add, next_state_add, done_add)
        self.learn_step=(self.learn_step+1)%learn_step_
        
        if len(self.memory) > BATCH_SIZE:
         if self.learn_step==0:
            experiences = self.memory.sample()
            for i in range(learntimes):
             self.learn(experiences, GAMMA)
             self.soft_update(self.critic_local, self.critic_target, TAU)
             self.soft_update(self.actor_local_first, self.actor_target_first, TAU)
             self.soft_update(self.actor_local_second,self.actor_target_second,TAU)

    def act(self, state, add_noise=True):
        """Returns actions for given state as per current policy."""
        state=state.reshape(1,-1)
        #state_first = torch.from_numpy(state[:,:24]).float().unsqueeze(0).to(device)
        #state_second=torch.from_numpy(state[:,24:48]).float().unsqueeze(0).to(device)
        state_first = torch.from_numpy(state[:,:24]).float().to(device)
        state_second=torch.from_numpy(state[:,24:48]).float().to(device)
        self.actor_local_first.eval()
        self.actor_local_second.eval()
        with torch.no_grad():
            action_first = self.actor_local_first(state_first).cpu().data.numpy()
            action_second=self.actor_local_second(state_second).cpu().data.numpy()
        self.actor_local_first.train()
        self.actor_local_second.train()
        if add_noise:
            action_first+= self.noise.sample()
            action_second+=self.noise.sample()
        action=np.zeros((2,2))
        action[0]=action_first
        action[1]=action_second
        return np.clip(action, -1, 1)

    def reset(self):
        self.noise.reset()

    def learn(self, experiences, gamma):
        """Update policy and value parameters using given batch of experience tuples.
        Q_targets = r + γ * critic_target(next_state, actor_target(next_state))
        where:
            actor_target(state) -> action
            critic_target(state, action) -> Q-value

        Params
        ======
            experiences (Tuple[torch.Tensor]): tuple of (s, a, r, s', done) tuples 
            gamma (float): discount factor
        """
        states, actions, rewards, next_states, dones = experiences
        
        # ---------------------------- update critic ---------------------------- #
        # Get predicted next-state actions and Q values from target models
        
        actions_next_first = self.actor_target_first(next_states[:,:24])
        actions_next_second=self.actor_target_second(next_states[:,24:48])
        actions_next=torch.cat((actions_next_first,actions_next_second),dim=1)
        Q_targets_next = self.critic_target(next_states, actions_next)

        
        # Compute Q targets for current states (y_i)
        
        Q_targets = rewards + (gamma * Q_targets_next * (1 - dones[0]))
        
        # Compute critic loss
        
        Q_expected = self.critic_local(states, actions)
        critic_loss = F.mse_loss(Q_expected, Q_targets)
        
        # Minimize the loss
        self.critic_optimizer.zero_grad()
        critic_loss.backward()
        torch.nn.utils.clip_grad_norm_(self.critic_local.parameters(), 1)
        self.critic_optimizer.step()

        # ---------------------------- update actor ---------------------------- #
        # Compute actor loss
        actions_pred_first = self.actor_local_first(states[:,:24])
        actions_pred_second = self.actor_local_second(states[:,24:48])
        actions_pred=torch.cat((actions_next_first,actions_next_second),dim=1)
        actor_loss = -self.critic_local(states, actions_pred).mean()
        # Minimize the loss
        self.actor_optimizer_first.zero_grad()
        actor_loss.backward(retain_graph=True)
        self.actor_optimizer_first.step()
        self.actor_optimizer_second.zero_grad()
        actor_loss.backward()
        self.actor_optimizer_second.step()
        
    def soft_update(self, local_model, target_model, tau):
        """Soft update model parameters.
        θ_target = τ*θ_local + (1 - τ)*θ_target

        Params
        ======
            local_model: PyTorch model (weights will be copied from)
            target_model: PyTorch model (weights will be copied to)
            tau (float): interpolation parameter 
        """
        for target_param, local_param in zip(target_model.parameters(), local_model.parameters()):
            target_param.data.copy_(tau*local_param.data + (1.0-tau)*target_param.data)

class OUNoise:
    """Ornstein-Uhlenbeck process."""

    def __init__(self, size, seed, mu=0., theta=0.15, sigma=0.2):
        """Initialize parameters and noise process."""
        self.size=size
        self.mu = mu * np.ones(size)
        self.theta = theta
        self.sigma = sigma
        self.seed = random.seed(seed)
        self.reset()

    def reset(self):
        """Reset the internal state (= noise) to mean (mu)."""
        self.state = copy.copy(self.mu)

    def sample(self):
        """Update internal state and return it as a noise sample."""
        x = self.state
        dx = self.theta * (self.mu - x) + self.sigma * np.random.standard_normal(self.size)
        self.state = x + dx
        return self.state

class ReplayBuffer:
    """Fixed-size buffer to store experience tuples."""

    def __init__(self, action_size, buffer_size, batch_size, seed):
        """Initialize a ReplayBuffer object.
        Params
        ======
            buffer_size (int): maximum size of buffer
            batch_size (int): size of each training batch
        """
        self.action_size = action_size
        self.memory = deque(maxlen=buffer_size)  # internal memory (deque)
        self.batch_size = batch_size
        self.experience = namedtuple("Experience", field_names=["state", "action", "reward", "next_state", "done"])
        self.seed = random.seed(seed)
    
    def add(self, state, action, reward, next_state, done):
        """Add a new experience to memory."""
        e = self.experience(state, action, reward, next_state, done)
        self.memory.append(e)
    
    def sample(self):
        """Randomly sample a batch of experiences from memory."""
        experiences = random.sample(self.memory, k=self.batch_size)

        states = torch.from_numpy(np.vstack([e.state for e in experiences if e is not None])).float().to(device)
        actions = torch.from_numpy(np.vstack([e.action for e in experiences if e is not None])).float().to(device)
        rewards = torch.from_numpy(np.vstack([e.reward for e in experiences if e is not None])).float().to(device)
        next_states = torch.from_numpy(np.vstack([e.next_state for e in experiences if e is not None])).float().to(device)
        dones = torch.from_numpy(np.vstack([e.done for e in experiences if e is not None]).astype(np.uint8)).float().to(device)

        return (states, actions, rewards, next_states, dones)

    def __len__(self):
        """Return the current size of internal memory."""
        return len(self.memory)

In [16]:
total_agent=2
agent = Agent(state_size=24, action_size=2, random_seed=6)

def ddpg(n_episodes=10, max_t=10, print_every=100):
    scores_deque = deque(maxlen=print_every)
    scores = []
    for i_episode in range(1, n_episodes+1):
        env_info = env.reset(train_mode=True)[brain_name]
        state = env_info.vector_observations
        score = np.zeros(total_agent)
        while True:
            action = agent.act(state)
            env_info =env.step(action)[brain_name] 
            next_state = env_info.vector_observations  
            reward = env_info.rewards
            done = env_info.local_done
            agent.step(state, action, reward, next_state, done)
            state = next_state
            score += reward
            if np.any(done):
                break 
        scores_deque.append(np.max(score))
        scores.append(np.max(score))
        print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_deque)), end="")
        torch.save(agent.actor_local_first.state_dict(), 'checkpoint_actor_first.pth')
        torch.save(agent.actor_local_second.state_dict(), 'checkpoint_actor_second.pth')
        torch.save(agent.critic_local.state_dict(), 'checkpoint_critic.pth')
        if i_episode % print_every == 0:
            print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_deque)))
            
    return scores

scores = ddpg()



Episode 9	Average Score: 0.00

RuntimeError: Trying to backward through the graph a second time, but the buffers have already been freed. Specify retain_graph=True when calling backward the first time.